# МТС Geohack.112

# Идея

Для человека не составит труда определить по спутниковому снимку плотность застройки, примерную численость населения в данном квадрате, или, например, торговый центр в котором в выходной день всегда много народа. Все эти признаки прямо или косвенно влияют на количество экстренных вызовов из данной области. Однако, есть еще и другие признаки, такие как количество аварий на данном перекрестке, который можно узнать только из статистики за некоторый период. Поэтому у меня возникла идея обьединить^ как информацию по спутниковым снимкам, так и информацию из различных текстовых источников.

Это можно сделать различными способами:
- Сделать нейросеть, в которой будет два входа - на один будет подаваться ебмединг из картинки (VGG,ResNet), а на другой признаки полученные из различных текстовых источников (OSM, data.mos и т.д.).
- Научить сеть предсказывать число звонков только на картинках, и использовать ее предсказания как дополнительные признаки вместе с текстовыми в каких-нибудь деревянных моделях.

Сразу хочу сказать, что оба подхода давали свои плоды, но в данном примере будет расмотрен первый, как наиболее краткий. 
    
   1) [Загрузка и обработка текстовых данных](#1)
   
   2) [Загрузка спутниковых снимков](#2)
   
   3) [Тренировка нейросети](#3)
   
   4) [Предсказания нейросети](#4)
   
   5) [Заключение](#5)

Картинка для привлечения внимания! Снизу представлены две области, на первой из них сетка дала достаточно низкий результат по звонкам а на второй очень высокий =) 

<img src="least_call.png" width="300">
<img src="most_call.png" width="300"> 

# 1) Загрузка текстовых данных <a name='1'></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import pandas 

df_zones = pandas.read_csv('data/zones.csv', index_col='zone_id')
df_zones.head()

,lat_bl,lon_bl,lat_tr,lon_tr,lat_c,lon_c,is_test,is_target,calls_daily,calls_workday,calls_weekend,calls_wd0,calls_wd1,calls_wd2,calls_wd3,calls_wd4,calls_wd5,calls_wd6
zone_id,,,,,,,,,,,,,,,,,,
0,55.378220,36.999105,55.382744,37.007050,55.380482,37.003077,0,0.0,0.002177,0.001325,0.004213,0.003006,0.000000e+00,6.885802e-04,0.003006,0.000000e+00,0.001374,0.007052
1,55.378252,37.006994,55.382775,37.014938,55.380514,37.010966,0,0.0,0.000996,0.000535,0.002096,0.000765,0.000000e+00,1.176913e-03,0.000765,0.000000e+00,0.001374,0.002818
2,55.378284,37.014883,55.382806,37.022827,55.380545,37.018855,0,0.0,0.001284,0.000724,0.002622,0.001322,0.000000e+00,1.020885e-03,0.001322,0.000000e+00,0.001374,0.003871
3,55.378315,37.022772,55.382837,37.030715,55.380576,37.026744,0,0.0,0.000968,0.000476,0.002143,0.000815,0.000000e+00,7.802124e-04,0.000815,0.000000e+00,0.001374,0.002912
4,55.378345,37.030662,55.382867,37.038603,55.380606,37.034633,0,0.0,0.000842,0.000310,0.002113,0.000783,1.821651e-07,7.286604e-07,0.000782,8.602241e-07,0.001374,0.002851


##### Прочитаем только точки имеющие теги из нужного нам региона
Чтение может занять 10–30 минут.

In [3]:
# %%time

# import osmread
# from tqdm import tqdm_notebook

# LAT_MIN, LAT_MAX = 55.309397, 56.13526
# LON_MIN, LON_MAX = 36.770379, 38.19270

# osm_file = osmread.parse_file('osm/RU-MOS.osm.pbf')
# tagged_nodes = [
#     entry
#     for entry in tqdm_notebook(osm_file, total=18976998)
#     if isinstance(entry, osmread.Node)
#     if len(entry.tags) > 0
#     if (LAT_MIN < entry.lat < LAT_MAX) and (LON_MIN < entry.lon < LON_MAX)
# ]

In [4]:
# import pickle

# with open('osm/tagged_nodes.pickle', 'wb') as fout:
#     pickle.dump(tagged_nodes, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle
tagged_nodes = pickle.load(open('osm/tagged_nodes.pickle', 'rb'))

## Построение таблицы признаков

На основе координат квадратов и данных из OpenStreetMap построим таблицу с простыми признаками, которые можно использовать для предсказания.

In [6]:
import collections 

df_features = collections.OrderedDict([])

##### Расстояние до Кремля

In [7]:
import math

kremlin_lat, kremlin_lon = 55.753722, 37.620657

def dist_calc(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

df_features['distance_to_kremlin'] = df_zones.apply(
    lambda row: dist_calc(row.lat_c, row.lon_c, kremlin_lat, kremlin_lon), axis=1)

##### Статистика по точкам из OSM

In [8]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# набор фильтров точек, по которым будет считаться статистика
POINT_FEATURE_FILTERS = [
    ('tagged', lambda 
     node: len(node.tags) > 0),
    ('railway', lambda node: node.tags.get('railway') == 'station'),
    ('shop', lambda node: 'shop' in node.tags),
    ('public_transport', lambda node: 'public_transport' in node.tags),
]

# центры квадратов в виде матрицы
X_zone_centers = df_zones[['lat_c', 'lon_c']].as_matrix()

for prefix, point_filter in POINT_FEATURE_FILTERS:

    # берем подмножество точек в соответствии с фильтром
    coords = np.array([
        [node.lat, node.lon]
        for node in tagged_nodes
        if point_filter(node)
    ])

    # строим структуру данных для быстрого поиска точек
    neighbors = NearestNeighbors().fit(coords)
    
    # признак вида "количество точек в радиусе R от центра квадрата"
    for radius in [0.001, 0.003, 0.005, 0.007, 0.01]:
        dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
        df_features['{}_points_in_{}'.format(prefix, radius)] = np.array([len(x) for x in inds])

    # признак вида "расстояние до ближайших K точек"
    for n_neighbors in [i**2 for i in range(1,5)]:
        dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
        df_features['{}_mean_dist_k_{}'.format(prefix, n_neighbors)] = dists.mean(axis=1)
        df_features['{}_max_dist_k_{}'.format(prefix, n_neighbors)] = dists.max(axis=1)
        df_features['{}_std_dist_k_{}'.format(prefix, n_neighbors)] = dists.std(axis=1)

    # признак вида "расстояние до ближайшей точки"
    df_features['{}_min'.format(prefix)] = dists.min(axis=1)

### Статистика из данных с https://data.mos.ru

Камеры в местах массового скопления людей

In [9]:
pd_camera_mass = pandas.read_json('extend_cool_data/data-8174-2018-03-19.json')

In [10]:
pd_camera_mass.head(2)

,Address,AdmArea,District,ID,OVDAddress,OVDPhone,Photo,UNOM,geoData,global_id
0,"2-� ���������� ������, ��� 5, �������� 25",�������� ���������������� �����,����� �������,MMC_SAO_0110_0,"�������� �����,�����18��","[{'PhoneOVD': '(495) 601-05-32'}, {'PhoneOVD':...",2fcd6bbc-f6a1-41bc-adaf-c85e40dbe6f9,2117289.0,"{'type': 'Point', 'coordinates': [37.553226, 5...",61213463
1,"����� 800-����� ������, ��� 4, ������ 1",�������� ���������������� �����,��������������� �����,MMC_SAO_0118_0,"����������� �����,�����8/2","[{'PhoneOVD': '(495) 601-05-34'}, {'PhoneOVD':...",12b79686-c94e-427f-b872-7c9817aa14a6,4863.0,"{'type': 'Point', 'coordinates': [37.5349, 55....",61213478


In [11]:
pd_camera_mass.geoData = pd_camera_mass.geoData.apply(lambda x: x['coordinates'][::-1])
coords_mass_camera = pd_camera_mass.geoData.tolist()
# строим структуру данных для быстрого поиска точек
neighbors = NearestNeighbors().fit(coords_mass_camera)

# признак вида "количество точек в радиусе R от центра квадрата"
for radius in [0.001*i for i in range(1,11)]:
    dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
    df_features['{}_points_in_{}'.format('mass_camera', radius)] = np.array([len(x) for x in inds])

# признак вида "расстояние до ближайших K точек"
for n_neighbors in [i for i in range(1,11)]:
    dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
    df_features['{}_mean_dist_k_{}'.format('mass_camera', n_neighbors)] = dists.mean(axis=1)
    df_features['{}_max_dist_k_{}'.format('mass_camera', n_neighbors)] = dists.max(axis=1)
    df_features['{}_std_dist_k_{}'.format('mass_camera', n_neighbors)] = dists.std(axis=1)

# признак вида "расстояние до ближайшей точки"
df_features['{}_min'.format('mass_camera')] = dists.min(axis=1)

### Статистика из данных по сотовым вышкам  https://opencellid.org

Эти данные были предварительно осортированы, чтобы получить базовые станции лежащие в Москве

In [12]:
pd_cellular = pandas.read_csv('extend_cool_data/moscow_cellular_towers.csv')

In [13]:
pd_cellular.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,GSM,250,35,7744,35168,0,37.748337,55.639572,1000,1,1,1459814853,1459779138,0
1,GSM,250,1,6402,29652,0,37.638832,55.740240,1000,23,1,1459707933,1486188885,0
2,GSM,250,2,7799,17093,0,37.592812,55.781708,1000,4,1,1459810852,1483019363,0
3,GSM,250,35,7711,33918,0,37.484665,55.617599,1000,1,1,1459814853,1459779138,0
4,GSM,250,2,5058,10360,0,37.891159,55.710297,1000,2,1,1459814852,1461557623,0


In [14]:
coords_cell = np.vstack([pd_cellular.lat.values,pd_cellular.lon.values]).T

In [15]:
# строим структуру данных для быстрого поиска точек
neighbors = NearestNeighbors().fit(coords_cell)

# признак вида "количество точек в радиусе R от центра квадрата"
for radius in [0.001*i for i in range(1,11)]:
    dists, inds = neighbors.radius_neighbors(X=X_zone_centers, radius=radius)
    df_features['{}_points_in_{}'.format('cell_tower', radius)] = np.array([len(x) for x in inds])

# признак вида "расстояние до ближайших K точек"
for n_neighbors in [i**2 for i in range(1,11)]:
    dists, inds = neighbors.kneighbors(X=X_zone_centers, n_neighbors=n_neighbors)
    df_features['{}_mean_dist_k_{}'.format('cell_tower', n_neighbors)] = dists.mean(axis=1)
    df_features['{}_max_dist_k_{}'.format('cell_tower', n_neighbors)] = dists.max(axis=1)
    df_features['{}_std_dist_k_{}'.format('cell_tower', n_neighbors)] = dists.std(axis=1)

# признак вида "расстояние до ближайшей точки"
df_features['{}_min'.format('cell_tower')] = dists.min(axis=1)

##### Итоговый набор признаков по квадратам

In [16]:
df_features = pandas.DataFrame(df_features, index=df_zones.index)
df_features.to_csv('data/features.csv')
df_features.head()

,distance_to_kremlin,tagged_points_in_0.001,tagged_points_in_0.003,tagged_points_in_0.005,tagged_points_in_0.007,tagged_points_in_0.01,tagged_mean_dist_k_1,tagged_max_dist_k_1,tagged_std_dist_k_1,tagged_mean_dist_k_4,...,cell_tower_mean_dist_k_64,cell_tower_max_dist_k_64,cell_tower_std_dist_k_64,cell_tower_mean_dist_k_81,cell_tower_max_dist_k_81,cell_tower_std_dist_k_81,cell_tower_mean_dist_k_100,cell_tower_max_dist_k_100,cell_tower_std_dist_k_100,cell_tower_min
zone_id,,,,,,,,,,,,,,,,,,,,,
0,56.852292,0,1,2,2,2,0.001845,0.001845,0.0,0.009619,...,0.045736,0.087004,0.020596,0.055600,0.096274,0.026508,0.064281,0.104057,0.029853,0.010764
1,56.511876,0,0,0,0,2,0.007528,0.007528,0.0,0.011118,...,0.047297,0.083238,0.020296,0.056922,0.098155,0.026037,0.065536,0.105729,0.029435,0.011759
2,56.173822,0,0,0,0,0,0.010501,0.010501,0.0,0.015790,...,0.049448,0.086839,0.019475,0.058785,0.098097,0.025102,0.067085,0.105977,0.028377,0.016886
3,55.838172,0,0,0,0,0,0.015280,0.015280,0.0,0.022225,...,0.052222,0.089035,0.018868,0.061034,0.096947,0.023965,0.068653,0.103041,0.026710,0.016001
4,55.504970,0,0,0,0,0,0.021936,0.021936,0.0,0.027709,...,0.055194,0.090025,0.018220,0.063425,0.096824,0.022764,0.070202,0.100069,0.024814,0.012639


# 2) Загрузка "спутниковых" снимков <a name='2'></a>

Итоговые картинки имеют размер 512х512 и физические размеры квадрата со стороной около 500 метров

In [17]:
from PIL import Image
import urllib
import os

def load_img(x,y,i):

    width, height = 512,512
    #Create a new image of the size require
    map_img = Image.new('RGB', (width,height))
    
    url = 'https://maps.googleapis.com/maps/api/staticmap?maptype=satellite&center={},{}&zoom=16&scale=2&size=512x512&key=your_api_key'.format(str(x),str(y))
    
    current_tile = str(x)+'-'+str(y)
    urllib.request.urlretrieve(url, current_tile)
    
    im = Image.open(current_tile)
    map_img.paste(im)
    os.remove(current_tile)
    map_img.save("photos/{}.png".format(i))
    

В данном примере я обкачиваю только квадраты, которые имеют target == 1. Можно также попробовать обкачать все данные и использовать их в обучении.

In [18]:
df_zones_for_nn = df_zones.query('is_test == 0 & is_target == 1 | is_test == 1')

In [19]:
import tqdm

for i in tqdm.tqdm(df_zones_for_nn.iterrows()):
    load_img(i[1][4],i[1][5],str(i[0]))
    

0it [00:00, ?it/s]


# 3) Тренировка нейросети <a name='3'></a>

В качестве таргета может выступать как среднее количество звонков по всем дням недели, так и отдельно по рабочим и выходным дням(два выхода или даже две отдельные модели). Также можно сделать 7 выходов по всем дням недели. В данном примере будет рассмотрены только среднее количество звонков по всем дням.

In [20]:
X = df_zones.query('is_test == 0 & is_target == 1').index.values
print(X.shape)
y =  df_zones.query('is_test == 0 & is_target == 1')[['calls_daily']].values

(4374,)


In [46]:
# преобразование таргет переменной
y = np.log1p(y)

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3936,)
(3936, 1)
(438,)
(438, 1)


In [49]:
from PIL import Image
from keras.utils import Sequence
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.5), # vertically flip 50% of all images
])


def load_img_as_array(img_path):
    
    img = Image.open('photos/'+img_path+'.png')
    
    return np.asarray(img)


class DataGen(Sequence):
    def __init__(self, xx, yy, data, batch_size):
        self.xx = xx
        self.yy = yy
        self.batch_size = batch_size
        self.data = data
    
    def __len__(self):
        return len(self.xx) // self.batch_size
    
    def __getitem__(self, i):
        X_list = []
        for j in range(self.batch_size):
            
                x = load_img_as_array(str(self.xx[i*self.batch_size + j]))
                X_list.append(x)
        
        X_batch = np.array(X_list)
        y_batch = self.yy[i*self.batch_size:(i+1)*self.batch_size]
        data_batch = self.data.loc[self.xx[i*self.batch_size:(i+1)*self.batch_size],:].values
        images_aug = seq.augment_images(X_batch)
        
        return [images_aug, data_batch], y_batch

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [52]:
from keras.models import Model,Input
from keras.layers import Dense, Dropout, Reshape, Concatenate
from keras.applications.resnet50 import ResNet50

def create_model(crop_size,t_f_size):
    
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(crop_size, crop_size, 3),pooling=None)    
    
    output_1 = base_model.output
    output_1 = Reshape((-1,))(output_1)
    
    input_2 = Input(shape=(t_f_size,))
    
    x =  Concatenate(axis=1)([input_2,output_1])
    x = Dense(2048, activation='relu', name='fc1')(x)
    x = Dropout(0.3, name='dropout_fc1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.3, name='dropout_fc2')(x)
    x = Dense(64, activation='relu', name='fc3')(x)
    x = Dropout(0.3, name='dropout_fc3')(x)
    predictions = Dense(1, activation="linear", name='predictions')(x)

    model = Model(inputs=[base_model.input,input_2], outputs=predictions)    
    #model.summary()
    
    return model

In [53]:
CROP_SIZE = 512

model = create_model(CROP_SIZE,t_f_size = df_features.shape[1])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.optimizers import Adam

model_folder = './models_experiment'
model_name = 'calls_daily_nn_resnet50'
batch_size = 8

opt = Adam(lr=1e-4)
model.compile(optimizer=opt, loss='mse', metrics=['mse'])

monitor = 'val_mean_squared_error'
model_path = os.path.join(model_folder, model_name + '-epoch-{epoch:03d}' + '-val_mse-{val_mean_squared_error:.3f}' + '.hdf5')

reduce_lr = ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=1, min_lr=1e-9, epsilon = 0.00001, verbose=1)
model_checkpoint = ModelCheckpoint(model_path, monitor=monitor, save_best_only=True, verbose=0)
early_stopping = EarlyStopping(monitor='val_mean_squared_error', patience=5, verbose=0)


callbacks = [reduce_lr, early_stopping, model_checkpoint]

history = model.fit_generator(
    generator=DataGen(X_train, y_train, df_features.loc[X_train, :], batch_size),
    steps_per_epoch=len(X_train)//batch_size,
    epochs=1000,
    
    validation_data=DataGen(X_test, y_test, df_features.loc[X_test, :], batch_size),
    validation_steps=len(X_test)//batch_size,

    callbacks=callbacks,
    max_queue_size = 10,
    workers=4)

# 4) Предсказания модели <a name='4'></a>

Валидация

In [ ]:
from keras.models import load_model
import glob

model_folder = './models_experiment'

models = glob.glob(os.path.join(model_folder, 'calls_daily_nn_resnet50*.hdf5'))

print(sorted(models)[-1])
loaded_model = load_model(sorted(models)[-1])
print("Loaded model from disk")

In [ ]:
pred_val = loaded_model.predict_generator(DataGen(X_test, y_test, df_features.loc[X_test, :], 1))

In [42]:
from scipy.stats import kendalltau

valid_score = kendalltau(y_test, pred_val).correlation
print('Validation score:', valid_score)

Validation score: 0.6586098635104689


Предсказания для теста

In [38]:
idx_test = df_zones.query('is_test == 1').index

pred_test = loaded_model_bow.predict_generator(DataGen(idx_test, idx_test, df_features.loc[idx_test, :], 1),verbose = 1)

8473/8473 [==============================] - 386s 46ms/step


In [39]:
pred_test = pred_test.reshape(-1)

In [ ]:
target_columns = ['calls_wd{}'.format(d) for d in range(7)]
df_test_predictions = pandas.DataFrame(collections.OrderedDict([
    (column_name, pred_test)
    for column_name in target_columns
]), index=idx_test)

df_test_predictions.to_csv('sample_submission.csv')
df_test_predictions.head(5)

# 5) Заключение <a name='5'></a>

Был рассмотрен один из возможных подходов использования спутниковых снимков для предсказания количества экстренных вызовов из заданного квадрата. Можно поэкспериментировать и попробовать следующее:
- Другая архитектруа нейросети
- Добавить еще других текстовых признаков
- Другая функция потерь (MAE или даже попробовать ранжирование)
- Докинуть различных аугментаций (сейчас только горизонтальные и вертикальные флипы)
- Другое приобразование целевой переменной (Бокса-Кокса)
- Получать предсказания только из картинок, и использовать как признак наравне с текстовымы, например, в LGBM

Мой нынешний скор 0.68 получен при использовании нейросетки над картинкам. Такой подход позволяет получить признаки, которые практически невозможно получить из текстовой информации.